In [1]:
from torchtext import datasets
from torchtext.data import to_map_style_dataset
import numpy as np
from torchtext.vocab import GloVe, vocab
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
import torch
import torch.nn as nn

# Load the dataset
train_iter, test_iter = datasets.AG_NEWS(split=('train', 'test'))

train_ds = to_map_style_dataset(train_iter)
test_ds = to_map_style_dataset(test_iter)


In [2]:
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer("basic_english")

my_vocab = build_vocab_from_iterator(map(lambda x: tokenizer(x[1]), train_ds), specials=['<pad>','<unk>'])
my_vocab.set_default_index(my_vocab["<unk>"])

In [3]:
'''from torch.utils.data import DataLoader
import torch
num_class = len(set([label for (label, _) in train_iter]))

text_pipeline = lambda x: my_vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

def collate_batch(batch):
    label_list, text_list = [], []
    for sample in batch:
        label, text = sample
        text_list.append(torch.tensor(text_pipeline(text), dtype=torch.long))
        label_list.append(label_pipeline(label))
    return torch.tensor(label_list, dtype=torch.long), torch.nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=my_vocab["<pad>"])

train_dataloader = DataLoader(
    train_iter, batch_size=64, shuffle=True, collate_fn=collate_batch
)

test_dataloader = DataLoader(
    test_iter, batch_size=64, shuffle=True, collate_fn=collate_batch
)'''

'from torch.utils.data import DataLoader\nimport torch\nnum_class = len(set([label for (label, _) in train_iter]))\n\ntext_pipeline = lambda x: my_vocab(tokenizer(x))\nlabel_pipeline = lambda x: int(x) - 1\n\ndef collate_batch(batch):\n    label_list, text_list = [], []\n    for sample in batch:\n        label, text = sample\n        text_list.append(torch.tensor(text_pipeline(text), dtype=torch.long))\n        label_list.append(label_pipeline(label))\n    return torch.tensor(label_list, dtype=torch.long), torch.nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=my_vocab["<pad>"])\n\ntrain_dataloader = DataLoader(\n    train_iter, batch_size=64, shuffle=True, collate_fn=collate_batch\n)\n\ntest_dataloader = DataLoader(\n    test_iter, batch_size=64, shuffle=True, collate_fn=collate_batch\n)'

In [3]:
glove_vectors = GloVe(name='6B', dim=300)


Vocab()


In [10]:
from torchtext.vocab import vocab
from collections import Counter, OrderedDict

unk_token = "<unk>"
unk_index = 0
glove_vocab = vocab(glove_vectors.stoi)
glove_vocab.insert_token("<unk>",unk_index)
glove_vocab.set_default_index(unk_index)


print(my_vocab['hello'])
print(glove_vocab['hello'])



12545
13075


In [59]:
import torch
import torch.nn as nn


class LSTMTextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class, pretrained_embeddings):
        super(LSTMTextClassificationModel, self).__init__()
        
        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=True)

        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_class)

    def forward(self, text):
        embedded = self.embedding(text)  
        lstm_out, _ = self.lstm(embedded)
        last_output = lstm_out[:, -1, :]
        output = self.fc(last_output)
        return output

vocab_size = len(my_vocab)  
embed_dim = 300
hidden_dim = 64
num_class = 4



pretrained_embeddings = glove_vectors.vectors
pretrained_embeddings = torch.cat((torch.zeros(1,pretrained_embeddings.shape[1]),pretrained_embeddings))
glove_model = LSTMTextClassificationModel(vocab_size, embed_dim, hidden_dim, num_class, pretrained_embeddings)




In [62]:
import time

# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

def train(dataloader):
    model.train()
    total_acc, total_count, max_acc = 0, 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| {:5d} batches '
                  '| accuracy {:8.3f}'.format(idx, total_acc / total_count))

            if max_acc < total_acc / total_count:
                max_acc = total_acc / total_count
                
            total_acc, total_count = 0, 0
            start_time = time.time()
    return max_acc


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

AttributeError: 'Tensor' object has no attribute 'parameters'

In [46]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()

    accu_train = train(train_dataloader)
    accu_val = evaluate(test_dataloader)

    #if accu_train > accu_val:
    #    scheduler.step()
    
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

|   500 batches | accuracy    0.255
|  1000 batches | accuracy    0.250
|  1500 batches | accuracy    0.258
-----------------------------------------------------------
| end of epoch   1 | time: 49.71s | valid accuracy    0.252 
-----------------------------------------------------------
